In [2]:
%matplotlib inline

import folium
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import nltk
import numpy as np
import operator 
import os
import pandas as pd
import pytz
import random
import re
import seaborn as sns
import string
import sys
import time
import vincent

from collections import Counter
from collections import defaultdict
from datetime import datetime
from matplotlib import dates
from matplotlib import rcParams
from matplotlib.ticker import MaxNLocator
from mpltools import style
from nltk import FreqDist
from nltk import bigrams 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from os import path
from pandas.tseries.resample import TimeGrouper
from pandas.tseries.offsets import DateOffset
from scipy.misc import imread
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

nltk.download('punkt')
nltk.download('mac_morpho')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/thiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package mac_morpho to
[nltk_data]     /Users/thiago/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
style.use('ggplot')
rcParams['axes.labelsize'] = 9
rcParams['xtick.labelsize'] = 9
rcParams['ytick.labelsize'] = 9
rcParams['legend.fontsize'] = 7
# rcParams['font.serif'] = ['Computer Modern Roman']
rcParams['font.serif'] = ['Ubuntu']
rcParams['text.usetex'] = False
rcParams['figure.figsize'] = 20, 10

# pd.set_option('display.max_colwidth', 200)
# pd.options.display.mpl_style = 'default'
# matplotlib.style.use('ggplot')
# sns.set_context('talk')
# sns.set_style('whitegrid')

print 'OK!'

OK!


* * *

### Testando *folium* e *data frames*

In [4]:
df = pd.read_json("data/big-data-fixed.json")
print 'OK!'
# df.info()

OK!


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503017 entries, 0 to 503016
Data columns (total 31 columns):
contributors                 0 non-null float64
coordinates                  430 non-null object
created_at                   503017 non-null datetime64[ns]
entities                     503017 non-null object
extended_entities            223637 non-null object
favorite_count               503017 non-null int64
favorited                    503017 non-null bool
filter_level                 503017 non-null object
geo                          430 non-null object
id                           503017 non-null int64
id_str                       503017 non-null int64
in_reply_to_screen_name      6889 non-null object
in_reply_to_status_id        4927 non-null float64
in_reply_to_status_id_str    4927 non-null float64
in_reply_to_user_id          6889 non-null float64
in_reply_to_user_id_str      6889 non-null float64
is_quote_status              503017 non-null bool
lang                

In [8]:
df['created_at'] = pd.to_datetime(pd.Series(df['created_at']))

df.set_index('created_at', drop=False, inplace=True)

# df.info()
df.index = df.index.tz_localize('GMT')
df.index = df.index - DateOffset(hours = 3)
df.index

df.head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,filter_level,geo,id,...,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
created_at,,,,,,,,,,,,,,,,,,,,,
2016-04-17 09:13:02+00:00,NaN,None,2016-04-17 12:13:02,"{u'user_mentions': [], u'symbols': [], u'hasht...","{u'media': [{u'source_user_id': 383766749, u's...",0,False,low,None,721672818824462336,...,7.216728e+17,7.216728e+17,0,False,NaN,"<a href=""https://about.twitter.com/products/tw...",#VzlaRepudiaAlChavismo #FORAdilma #FORAlula #F...,2016-04-17 12:13:02.697,False,"{u'follow_request_sent': None, u'profile_use_b..."
2016-04-17 09:13:03+00:00,NaN,None,2016-04-17 12:13:03,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,721672820430925824,...,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Gente... Renan Calheiros? Cê jura?\n#Impeachme...,2016-04-17 12:13:03.080,False,"{u'follow_request_sent': None, u'profile_use_b..."
2016-04-17 09:13:02+00:00,NaN,None,2016-04-17 12:13:02,"{u'user_mentions': [], u'symbols': [], u'hasht...",{u'media': [{u'expanded_url': u'http://twitter...,0,False,low,None,721672818467987456,...,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",cara dos governistas qnd a dilma cair #Impeac...,2016-04-17 12:13:02.612,False,"{u'follow_request_sent': None, u'profile_use_b..."
2016-04-17 09:13:03+00:00,NaN,None,2016-04-17 12:13:03,"{u'user_mentions': [], u'symbols': [], u'hasht...",NaN,0,False,low,None,721672823090114562,...,NaN,NaN,0,False,NaN,"<a href=""http://www.twitter.com"" rel=""nofollow...",Hoje é um dia importante pro Brasil !!! Eu apo...,2016-04-17 12:13:03.714,False,"{u'follow_request_sent': None, u'profile_use_b..."
2016-04-17 09:13:03+00:00,NaN,None,2016-04-17 12:13:03,"{u'user_mentions': [{u'id': 3931855841, u'indi...","{u'media': [{u'source_user_id': 3931855841, u'...",0,False,low,None,721672823731789824,...,NaN,NaN,0,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/iphone"" r...",RT @Seu_Demo: Tudo pronto pro #ImpeachmentDay\...,2016-04-17 12:13:03.867,False,"{u'follow_request_sent': None, u'profile_use_b..."


In [55]:
tweets30s = df['created_at'].resample('1t', how='count')
tweets30s.head()

created_at
2016-04-17 09:13:00+00:00    186
2016-04-17 09:14:00+00:00    210
2016-04-17 09:15:00+00:00    201
2016-04-17 09:16:00+00:00    209
2016-04-17 09:17:00+00:00    232
Freq: T, Name: created_at, dtype: int64

In [56]:
avg = tweets30s.mean()

vincent.core.initialize_notebook()
area = vincent.Area(tweets30s)
area.colors(brew='Spectral')
area.display()

* * * 

### Funções de teste

In [ ]:
def datetimeify(df):
    df['created_at'] = pd.DatetimeIndex(df.created_at)
    return df

In [ ]:
def sentiment(df):
    text = df.dropna(subset=['text']).text
    sentiment = text.apply(lambda text: TextBlob(text).sentiment)
    df['polarity'] = sentiment.apply(lambda sentiment: sentiment.polarity)
    df['subjectivity'] = sentiment.apply(lambda sentiment: sentiment.subjectivity)
    return df

In [ ]:
def influence(df):
    internal = np.sqrt(df.user_followers_count.apply(lambda x: x + 1))
    external = np.sqrt(df.retweet_count.apply(lambda x: x + 1))
    df['influence'] = internal * external
    return df

In [ ]:
def influenced_polarity(df):
    df['influenced_polarity'] = df.polarity * df['influence']
    return df

In [ ]:
def georeference(df):
    def place_to_coordinate(place_str, kind):
        if pd.isnull(place_str):
            return float('nan')
        number_matcher = r'(-?\d+\.\d+)[,\]]'
        coordinates = re.findall(number_matcher, place_str)
        coordinate = tuple(float(n) for n in coordinates[:2])

        if kind == 'longitude':
            return coordinate[0]
        elif kind == 'latitude':
            return coordinate[1]
    df['latitude'] = df.place.apply(place_to_coordinate, kind='latitude')
    df['longitude'] = df.place.apply(place_to_coordinate, kind='longitude')

    return df

In [ ]:
def preprocess(df):
    return (df.pipe(datetimeify))

In [ ]:
def preprocess_df(df):
    cleaned = df.pipe(set_hashtags)
    copy = cleaned.copy()
    return preprocess(copy)


In [ ]:
def load_df(input_filename):
    raw_df = pd.read_json(input_filename)
    return preprocess(raw_df)

print 'OK'


* * *

### Horários de Tweets

In [ ]:
df = load_df('data-docker/peda.json')
df.head()

In [ ]:
tweets = pd.read_json('data-docker/peda.json')
tweets['created_at'] = pd.to_datetime(pd.Series(tweets['created_at']))

tweets.set_index('created_at', drop=False, inplace=True)

tweets.info()
tweets.index = tweets.index.tz_localize('GMT')
tweets.index = tweets.index - DateOffset(hours = 3)
tweets.index

tweets.head()

In [ ]:
tweets30s = tweets['created_at'].resample('1s', how='count')
tweets30s.head()

In [ ]:
avg = tweets30s.mean()

vincent.core.initialize_notebook()
area = vincent.Area(tweets30s)
area.colors(brew='Spectral')
area.display()

* * *

### Testes sentimental analysis

In [ ]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = "RT @medeirosthiiago: testando exemplo TCC! :D http://example.com #ImpeachmentDay"
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']
